In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

from eda.constants import FIGURES_DIR, FIG_SIZE
from polarization.utils import get_polarization, get_polarization_by_time
from preprocessing.utils import load_event_comments
from preprocessing.constants import OUTPUT_DIR, EVENTS, EVENTS_INFO

sns.set(rc={"figure.figsize": FIG_SIZE})

In [ ]:
event_polarization = {}

for event in EVENTS:
    print(e)
    event_comments = load_event_comments(event)
    event_polarization[event] = get_polarization(event, data)

    with open(f"{OUTPUT_DIR}/{event}_polarization_leaveout.json", "w") as f:
        json.dump(event_polarization[event], f)

    # Plot user polarization
    _, (dem_polarization, rep_polarization) = event_polarization[event]
    user_pols = pd.DataFrame(
        zip(
            dem_polarization + rep_polarization,
            ["dem"] * len(dem_polarization) + ["rep"] * len(dem_polarization),
        ),
        columns=["polarization", "affiliation"],
    )

    ax = sns.boxplot(y="polarization", x="affiliation", data=user_pols)

    ax.set_ylabel("User polarization")

    plt.savefig(
        fname=f"{FIGURES_DIR}/{event}_user_polarization.pdf",
        bbox_inches="tight",
        pad_inches=0,
        format="pdf",
    )

    plt.show()


In [ ]:
pol_by_week_events = {}

for event in EVENTS:
    pol_by_week_events[event] = get_polarization_by_time(event, data, freq="week")
    pol_by_week_events[event].to_csv(f"{OUTPUT_DIR}/{event}_polarization_by_week.csv")

In [ ]:
# Plot weekly

for event in EVENTS:
    ax = sns.lineplot(
        x=pol_by_week_events[event]["date"],
        y=pol_by_week_events[event]["polarization"],
        label="Polarization",
    )
    sns.lineplot(
        data=pol_by_week_events[event],
        x="date",
        y="random_polarization",
        label="Polarization with random user assignment",
    )

    plt.axvline(
        EVENTS_INFO[event]["date"],
        linestyle="--",
        color="red",
        label=f"{EVENTS_INFO[event]['name']} date",
    )

    plt.xlabel("Date")
    plt.ylabel("Polarization")
    plt.title("Weekly polarization score")
    plt.legend()
    plt.savefig(
        fname=f"{FIGURES_DIR}/{event}_polarization_by_week.pdf",
        bbox_inches="tight",
        pad_inches=0,
        format="pdf",
    )
    plt.show()


In [ ]:
pol_by_day_events = {}

for event in EVENTS:
    pol_by_day_events[event] = get_polarization_by_time(event, data, freq="day")
    pol_by_day_events[event].to_csv(f"{OUTPUT_DIR}/{event}_polarization_by_day.csv")

In [ ]:
# Plot daily

for event in EVENTS:
    ax = sns.lineplot(
        x=pol_by_day_events[event]["date"],
        y=pol_by_day_events[event]["polarization"],
        label="Polarization",
    )
    sns.lineplot(
        data=pol_by_day_events[event],
        x="date",
        y="random_polarization",
        label="Polarization with random user assignment",
    )

    plt.axvline(
        EVENTS_INFO[event]["date"],
        linestyle="--",
        color="red",
        label=f"{EVENTS_INFO[event]['name']} date",
    )

    plt.xlabel("Date")
    plt.ylabel("Polarization")
    plt.title("Daily polarization score")
    plt.legend()
    plt.savefig(
        fname=f"{FIGURES_DIR}/{event}_polarization_by_day.pdf",
        bbox_inches="tight",
        pad_inches=0,
        format="pdf",
    )
    plt.show()
